In [1]:
import numpy as np
import gmpy2
from gmpy2 import mpz,isqrt,is_prime
import math
import time

In [2]:
def is_smooth(n, factor_base):
    if n < 1:
        return False
    for i in factor_base:
        while n % i == 0:
            n //= i
    if n == 1:
        return True
    return False

def generate_factor_base(n, size_of_base):
    # if the size of base is not specified, calculate
    if size_of_base == 0:
        if n > 1.00e+14:
            size_of_base = min(gmpy2.exp((1/6) * (isqrt(mpz(gmpy2.log(n)*(gmpy2.log(n)))))), 150)
        elif n > 1.00e+8:
            size_of_base = min(gmpy2.exp((1/4) * (isqrt(mpz(gmpy2.log(n)*(gmpy2.log(n)))))), 120)
        elif n > 1.00e+5:
            size_of_base = min(gmpy2.exp((1/2) * (isqrt(mpz(gmpy2.log(n)*(gmpy2.log(n)))))), 70)
        else:
            size_of_base = min(gmpy2.exp((1/2) * (isqrt(mpz(gmpy2.log(n)*(gmpy2.log(n)))))), 30)
    base = [2]
    i = 3
    while len(base) < size_of_base:
        if is_prime(i):
            base += [i]
        i += 1
    return base

def get_factors(n, factor_base):
    exponents = np.zeros(len(factor_base))
    for i, prime in enumerate(factor_base):
        while n % prime == 0:
            # fun fact: using n = n / prime will cause over flow for numbers > 1e+24, so don't use that
            n //= prime
            exponents[i] += 1
    if n == 1:
        # return exponens and exponents mod 2 for the binary matrix
        return exponents, exponents % 2
    return None

In [3]:
def matrix_to_file(matrix):
    f = open("matrix_file.txt", "w")
    f.write(str(matrix.shape[0]) + " " + str(matrix.shape[1]) + "\n")
    for row in matrix:
        f.write(' '.join(str(int(i)) for i in row) + "\n")
    f.close

def solutions_from_file(number_of_relations):
    f = open("solutions.txt", "r")
    solutions = np.array([]).reshape(0,number_of_relations)
    next(f)
    for i in f:
        solutions = np.vstack([solutions, np.array(np.fromstring(i, dtype=int, sep=' '), ndmin=2)])
    return solutions

In [4]:
def generate_relatinos(n, factor_base):
    relations = np.array([]).reshape(0,2)
    matrix = np.array([]).reshape(0, len(factor_base))
    bound = 20
    while len(relations) < len(factor_base):
        for k in range(1, bound):
            for j in range(1, k + 1):
                # calculate r and r^2 mod N
                r = math.floor(math.sqrt(k * n)) + j
                r_sq_mod = (r**2) % n
                # check if r^2 mod N is a smooth number
                if is_smooth(r_sq_mod, factor_base):
                    exponents, row = get_factors(r_sq_mod, factor_base)
                    # check if there is not the same binary row already
                    if not any(np.equal(matrix,row).all(1)):
                        relations = np.vstack([relations, np.array([r, exponents], dtype=object)])
                        matrix = np.vstack([matrix, row])
                    if len(relations) >= len(factor_base) + 10:
                        return relations, matrix
            bound += 10
    return relations, matrix

In [5]:
def get_solutions(n, solutions, relations, factor_base):
    attempts_cnt = 0
    for solution in solutions:
        attempts_cnt += 1
        x = mpz(1)
        y_exp = np.zeros(int(len(factor_base)))
        # evaluate one solution 
        for i, elem in enumerate(solution):
            if elem == 1:
                x *= mpz(relations[i][0])
                y_exp += relations[i][1]
        y = mpz(1)
        # restore y from the factors
        for i in range(len(factor_base)):
            y *= mpz(factor_base[i] ** (y_exp[i] / 2))
        # calculate gcd and check the condition p != (-)1
        p = abs(int(gmpy2.gcd(mpz(y - x), mpz(n))))
        if p != mpz(1) and p != mpz(n):
            return p, mpz(n) / p, attempts_cnt
    return 0, 0, attempts_cnt

In [6]:
def quadratic_sieve_algorithm(n, size_of_factor_base = 0):
    # set a time starting point
    start_time = time.time()
    # generete factor base
    factor_base = generate_factor_base(n, size_of_factor_base)
    # generate realations x^2 = y mod N, where y == smooth number
    relations, matrix = generate_relatinos(n, factor_base)
    # load matrix into a file
    matrix_to_file(matrix)
    # run the Gaussian elimination in c++
    ! g++ -std=c++17 -Wall -pedantic GaussianElm.cpp
    ! ./a.out matrix_file.txt solutions.txt
    # get solutions from file
    solutions = solutions_from_file(matrix.shape[0])
    # try solution from gaussian elimination
    p, q, attemps = get_solutions(n, solutions, relations, factor_base)
    # print a out put
    print("\nResult:")
    print(str(int(n)) + " = " + str(int(p)) + " * " + str(int(q)))
    print("Number of attempts: " + str(attemps))
    print("Runtime: " + str(time.time() - start_time) + "s")
    # delete files for gaussian elimination
    ! rm matrix_file.txt | rm solutions.txt

In [8]:
#323
#307561
#31741649
#3205837387 
#392742364277
#58660100715217
#508111006563443
#1702818100334447
#13351810745855651
#94217663539251883
#140652271589468237
#3363271280049834261077
#170527948450228765165631
#106565238310234107615313

quadratic_sieve_algorithm(92434447339770015548544881401, 300)

KeyboardInterrupt: 